In [5]:

import numpy as np
from Read_AllFiles import ReadFiles
from numpy import save
from tqdm import tqdm
path = ('D:\\tuc\\exam10\\Thesis\\Dataset\\FIXED_CMU_BVH')
out = ('D:\\tuc\\exam10\\Thesis\\Dataset\\CMU_nmpy\\')
files=ReadFiles.getListOfFiles(None,path)

for file in tqdm(files):
    data = open(file, 'r')
    Lines = data.readlines()
    Hierarchy = Lines[:187]
    Motion = Lines[187:]
    arr = np.array(Motion)
    name= out +file.split("\\")[6].split(".bvh")[0]
    
    break
    np.savez_compressed(name, arr)


  0%|          | 0/1074 [00:00<?, ?it/s]


In [1]:
import numpy as np
import pickle
import glob
from numpy.lib.function_base import average
import pandas as pd
from Read_AllFiles import ReadFiles
from numpy import load
from numpy import save
from tqdm import tqdm




path = "D:\\tuc\\exam10\\Thesis\\Dataset\\CMU_nmpy"
out = "D:\\tuc\\exam10\\Thesis\\Dataset\\nmpy_Resized\\"
files=ReadFiles.getListOfFiles(None,path)
shape = []
for file in tqdm(files):
    data = load(file, allow_pickle=True)
    shape.append(data[data.files[0]].shape[0])
print(max(shape))
print(min(shape))
print(average(shape))
m = max(shape)

for file in tqdm(files):
    data = load(file, allow_pickle=False)
    d = data[data.files[0]]
    f= np.resize(d,(m,d.shape[1],d.shape[2]))
    name= out +file.split("\\")[6].split(".npz")[0]
    np.savez_compressed(name, f)

NameError: name 'm' is not defined

In [3]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras import utils
import numpy as np
import pickle
import glob
import pandas as pd
from Read_AllFiles import ReadFiles
from numpy import load
from tqdm import tqdm

###   (4320,150,13,2) ---->>>   (4320, 150, 26)

#X = np.reshape(X, (X.shape[0], X.shape[1],X.shape[2]*X.shape[3]))
#X= X/np.max(X)

#y = utils.to_categorical(y)


model = Sequential()
model.add(Input(shape=X.shape))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-biggeer.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode = 'min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs = 30, batch_size=32, callbacks=callbacks_list)


2021-12-11 16:03:22.983867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-11 16:03:22.983889: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-11 16:03:25.625365: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-11 16:03:25.625435: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kyriakos-GL72M-7RDX): /proc/driver/nvidia/version does not exist
2021-12-11 16:03:25.626643: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations: 

AttributeError: 'list' object has no attribute 'shape'

In [2]:
import numpy as np
import pickle
import glob
from numpy.lib.function_base import average
import pandas as pd
from Read_AllFiles import ReadFiles
from numpy import load
from numpy import save
from tqdm import tqdm
import itertools

path = "D:\\tuc\\exam10\\Thesis\\Dataset\\nmpy_Resized"

files=ReadFiles.getListOfFiles(None,path)
clips = []
for file in tqdm(files):
    data = load(file, allow_pickle=True)
    clips.append(data[data.files[0]])

name = "D:\\tuc\\exam10\\Thesis\\Dataset\\CMU_clips"
merged = np.asarray(clips)
print(merged.shape)
np.savez_compressed(name, merged)

100%|██████████| 1073/1073 [00:15<00:00, 70.14it/s]


(1073, 3198, 31, 6)


In [8]:


import glob
import os
import pandas as pd
import xlrd

loc = ("D:\\tuc\\exam10\\Thesis\Dataset\\cmu-mocap-master\\cmu-mocap-index-spreadsheet.xls")
dict = {}
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)
 
for i in range(12,sheet.nrows):
    l = []
    l.append(str(sheet.cell_value(i, 1)))
    l.append(str(sheet.cell_value(i, 2)))
    v = {sheet.cell_value(i, 0): l}
    dict.update(v)

list_search = ["walk"]
count=0

"""
for value in dict.copy().values():
    key = list(dict.keys())[list(dict.values()).index(value)]
    if any(i in value[0].lower() for i in list_search) or any(i in value[1].lower() for i in list_search) :
        count+=1
    else :
        dict.pop(key, None)

print(count)
print(len(dict))
"""

{'01_01': ['playground - forward jumps, turn around', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_02': ['playground - climb', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_03': ['playground - climb, hang, swing', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_04': ['playground - climb', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_05': ['playground - climb, go under', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_06': ['playground - climb, sit, dangle legs, descend', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_07': ['playground - climb, sit, dangle legs, jump down', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_08': ['playground - climb, sit, dangle legs, rock back, lower self to ground', 'Subject #1 (climb, swing, hang on playground equipment)'], '01_09': ['playground - climb, hang, hold self up with arms straight, swing, drop, sit, dangle legs, go under', 